In [ ]:
# selenium 불러오기
from selenium import webdriver
import time

# chrome창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 실행할 웹페이지 불러오기(구글 지도 예시)
driver.get("https://www.google.com/maps/")

# 선택자로 페이지 요소 선택하기(단일)
driver.find_element_by_css_selector("선택자")

# 선택자로 페이지 요소 선택하기(다중)
driver.find_elements_by_css_selector("선택자")

# 해당 요소에 문자 입력하기(ex. 버튼)
button.send_keys("텍스트")

# 해당 요소 클릭하기
button.click()

# 의도적으로 3초 쉬어주기
time.sleep(3)

# 여러 페이지(999)에서 반복하기
for i in range(999):
    # 시간 지연
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

        # 가게 주소 데이터 수집 // span.section-result-location
        addr = s.find_element_by_css_selector("span.section-result-location").text

        print(title, "/", score, "/", addr)

    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#n7lv7yjyC35__section-pagination-button-next")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import pymysql

query = '미용실' #검색어
sortingOrder = 'reviewCount' #정렬방식
arrLat = [] #위도 y
arrLng = [] #경도 x
lat = '35.179949125560846'
lng = '129.07495481148936'
location = "부산광역시청"

link = 'https://m.place.naver.com/hairshop/list?sm=mtp_hty.top&where=m&query='
link += query + '&x=' + lng + '&y=' + lat + '&level=top&entry=pll&sortingOrder=' + sortingOrder
link += '&gender=f&listViewQuery=' + query
        
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 네이버 플레이스 링크 접속
driver.get(link)
time.sleep(3)

last_height = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
hover = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(1) > div._15UxR > div > div > a._2QhH-._1-QNb")
hover.send_keys(Keys.ENTER)

while True:
    for i in range(30):
        hover.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    size = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
    if size['height'] == last_height['height']:
        time.sleep(20)
        if size['height'] == last_height['height']:
            print("데이터 로드 완료")
            break
        print('네트워크 쫌 느림 한번 더 실행')
    last_height = size
#     print(size['height'])

stores = driver.find_elements_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2) > ul > li > div._1WDkQ > div._1PT3q")
arrTitle = []
arrHref = []
for s in stores:
    title = s.find_element_by_css_selector("span._3t53y").text
    href = s.find_element_by_css_selector("a._1ptue").get_attribute('href')
    href = href.partition('hairshop/')[2].split('?')[0]
    arrTitle.append(title)
    arrHref.append(href)
# print(arrTitle)
# print(arrHref)

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)

cursor = conn.cursor()

# # 테이블생성
# sql = '''CREATE TABLE hairshop ( 
# c_idx int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, 
# c_idx_naver int(20),
# c_name varchar(255),
# c_search_location varchar(50)
# ) 
# ''' 
# cursor.execute(sql)

# SELECT
# sql = "SELECT * FROM `hair_commerce_tb`;"
# cursor.execute(sql)
# result = cursor.fetchall()
# print(result)

for title,href in zip(arrTitle, arrHref):
    sql = "INSERT INTO hairshop (c_name, c_idx_naver, c_search_location) VALUES (%s,%s,%s)"
    val = (title,href,location)
    cursor.execute(sql, val)

conn.commit()
conn.close()
print("데이터 저장 완료")
    
# 크롬창 닫기
#driver.close()

24146
36384
48181
59891
70622
79998
95737
109746
129415
141763
160016
178298
196051
213159
230467
248088
259904
276152
295115
309054
328046
341842
360303
372995
데이터 로드 완료
완료


In [ ]:
import requests
import time
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)
cursor = conn.cursor()

sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청';"
cursor.execute(sql)
result = cursor.fetchall()
# print(result[0][0]) #c_idx
# print(result[0][1]) #c_idx_naver
# print(result[0][2]) #c_name
# print(result[0][3]) #c_search_location
# print(len(result))

# row = []

for res in result:
    # c_phone=None; c_addr=None; c_hours=None; c_score=None; c_kind=None;
    c_home = []
    c_price = []
    c_designer = []

    #----------------------------home---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/home?entry=pll'.format(res[1])
    response = requests.get(url)

    if response.status_code == 200:
        html = response.content.decode('utf-8','replace')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            c_phone = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div.afDDM > div > span:nth-child(1) > a').attrs['href'].split(':')[1]
        except:
            print("{0}의 c_phone 못찾음".format(url))
            c_phone = None
        try:
            c_addr = soup.select_one('li._1M_Iz._1aj6- > div > span._2yqUQ').text
        except:
            print("{0}의 c_addr 못찾음".format(url))
            c_addr = None
        try:
            c_hours = soup.select_one('li._1M_Iz._2KHqk > div > div > div').text
        except:
            print("{0}의 c_hours 못찾음".format(url))
            c_hours = None
        try:
            c_score = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em').text
        except:
            print("{0}의 c_score 못찾음".format(url))
            c_score = None
        c_home.append((c_phone, c_addr, c_hours, c_score))

    else : 
        print("페이지 로드 실패 : {0}".format(url))
        print(response.status_code)

    # ----------------------------price---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/price?entry=pll'.format(res[1])
    response = requests.get(url)

    if response.status_code == 200:
        html = response2.content.decode('utf-8','replace')
        soup = BeautifulSoup(html, 'html.parser')

        try:
            element = soup.select('#app-root > div > div.place_detail_wrapper > div:nth-child(4) > div.place_section.no_margin > div > div > ul > li')
            for ele in element:
                try:
                    kind = ele.select_one('span._1kuzz').string
                except:
                    print("{0}의 kind 못찾음".format(url))
                    kind = None
                ele_li = ele.select('ul._12NKi > li')
                for li in ele_li:
                    try:
                        name = li.select_one('div > div > span > span').string
                    except:
                        print("{0}의 name 못찾음".format(url))
                        name = None
                    try:
                        price = li.select_one('em._12eac').text[0:-1]
                    except:
                        print("{0}의 price 못찾음".format(url))
                        price = None
                    c_price.append((res[0], kind, name, price))
        except:
            print("price의 element or ele_li 못 찾음 - {0}".format(url))

    else : 
        print("페이지 로드 실패 : {0}".format(url))
        print(response.status_code)

    #----------------------------stylist---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/stylist'.format(res[1])

    driver = webdriver.Chrome("./chromedriver")

    driver.get(url)
    time.sleep(3)

    try:
        more = driver.find_element_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div._2kAri > a")
        more.click()
        time.sleep(0.5)
    except:
        print("")
        
    designers = driver.find_elements_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div:nth-child(2) > ul > li')
    for designer in designers:
        try:
            d_idx_naver = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a').get_attribute('href').partition('stylist/')[2]
        except:
            print("{0}의 d_idx_naver 못찾음".format(url))
            d_idx_naver = None
        try:
            d_name = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a > span').text
        except:
            print("{0}의 d_name 못찾음".format(url))
            d_name = None
        try:
            d_comment = designer.find_element_by_css_selector('div._2TEwD > div._2KAFb > a').text
        except:
            print("{0}의 d_comment 못찾음".format(url))
            d_comment = None
        try:
            d_score = designer.find_element_by_css_selector('div._2TEwD > div._1Ezoe > a > span._3XrWs > em').text
        except:
            print("{0}의 d_score 못찾음".format(url))
            d_score = None
        try:
            d_reservation_link = designer.find_element_by_css_selector('div.OO9oN > a').get_attribute('href')
        except:
            print("{0}의 d_reservation_link 못찾음".format(url))
            d_reservation_link = None
        c_designer.append((d_idx_naver, d_name, d_score, d_reservation_link, d_comment))

    driver.close()


    # ----------------------------db저장---------------------------

    print(c_home)
    print(c_price)
    print(c_designer)
    print("'{0}'매장 데이터 저장완료".format(res[2]))
    print("--------------------------------------------------------------------------------------")
    



# conn = pymysql.connect(
#     charset='utf8',
#     user='root', 
#     passwd='', 
#     host='localhost',
#     db='test'
# )

# cursor = conn.cursor()

# # # 테이블생성
# # sql = '''CREATE TABLE hairshop ( 
# # c_idx int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, 
# # c_idx_naver int(20),
# # c_name varchar(255),
# # c_search_location varchar(50)
# # ) 
# # ''' 
# # cursor.execute(sql)

# # SELECT
# # sql = "SELECT * FROM `hair_commerce_tb`;"
# # cursor.execute(sql)
# # result = cursor.fetchall()
# # print(result)

# for title,href in zip(arrTitle, arrHref):
#     sql = "INSERT INTO hairshop (c_name, c_idx_naver, c_search_location) VALUES (%s,%s,%s)"
#     val = (title,href,location)
#     cursor.execute(sql, val)

# conn.commit()
# conn.close()
# print("데이터 저장 완료")
    
# # 크롬창 닫기
# #driver.close()

[('0507-1358-0886', '경남 창원시 성산구 마디미로 10 두바빌딩 2층', '매일 10:00 - 22:00 ', '4.94')]
[(1, '컷', '여성 컷', '18,000'), (1, '컷', '남성 컷', '22,000'), (1, '일반펌', '여성일반펌', '100,000'), (1, '일반펌', '남성일반펌', '70,000'), (1, '열펌', '여성열펌', '120,000'), (1, '클리닉', '여성크리닉', '70,000')]
[('3179468', '유주 디자이너 (화요일휴무)', '4.96', 'https://booking.naver.com/booking/13/bizes/266278/items/3179468?theme=place&area=ple', '고객맞춤형 디자이너 유주 ! 컬러는 물론 열펌까지 다 찾아주세용 :) * 특수컬러 예약시 요청사항에 기재부탁드립니당 ! *'), ('3179483', '엣지 디자이너 (목요일휴무)', '4.96', 'https://booking.naver.com/booking/13/bizes/266278/items/3179483?theme=place&area=ple', '어.,....네....잘부탁드립니다 맡겨주세요!!! ㅎㅎ'), ('3179467', '은희 디자이너 (화요일휴무)', '4.96', 'https://booking.naver.com/booking/13/bizes/266278/items/3179467?theme=place&area=ple', '편안함 속에 고객님만의 스타일을 찾아드리겠습니다. (네이버 예약 안될시에 매장으로 전화해주세요)'), ('3499018', '지오 디자이너 (화요일휴무)', '4.95', 'https://booking.naver.com/booking/13/bizes/266278/items/3499018?theme=place&area=ple', '고객님의 아름다움을 책임지는 지오입니다! 믿고 따라만와주세요!'), ('3179494', '혜민 디자이너 (월요


[('0507-1307-4469', '경남 창원시 성산구 마디미로73번길 25', '매일 10:00 - 22:00 ', '4.88')]
[(5, '컷', '여성 컷', '18,000'), (5, '컷', '남성 컷', '22,000'), (5, '일반펌', '여성일반펌', '100,000'), (5, '일반펌', '남성일반펌', '70,000'), (5, '열펌', '여성열펌', '120,000'), (5, '클리닉', '여성크리닉', '70,000')]
[('3776022', 'STYLIST 아인(화요일휴무)', '5', 'https://booking.naver.com/booking/13/bizes/475720/items/3776022?theme=place&area=ple', '퍼머맛집인증으로 니즈에맞는 스타일을 찾아드립니다~ 7월 오후근무 14:00~22:00'), ('3776005', 'STYLIST 현아(일요일휴무)', '4.98', 'https://booking.naver.com/booking/13/bizes/475720/items/3776005?theme=place&area=ple', '열펌 전문인 만큼 인생퍼머를 경험하실 꺼에요~ 7월 오전근무 10:00~18:00'), ('3776001', 'STYLIST 다운(월요일휴무)', '4.9', 'https://booking.naver.com/booking/13/bizes/475720/items/3776001?theme=place&area=ple', '당신에 기초하여, 아름다운 분위기를 선물해드리겠습니다 7월 오후근무 14:00~22:00'), ('3776021', 'STYLIST 채현(수요일휴무)', '4.96', 'https://booking.naver.com/booking/13/bizes/475720/items/3776021?theme=place&area=ple', '고객님의 니즈에 맞게 트렌디함으로 보답하겠습니다~ 7월 오후근무 14:00~22:00'), ('3868512', 'STYLIST 


[('0507-1327-1436', '경남 창원시 의창구 용지로 149 남강빌딩 1층 모티브헤어', '매일 10:00 - 20:30 명절당일휴뮤 ,유동적', '4.92')]
[(10, '컷', '여성 컷', '18,000'), (10, '컷', '남성 컷', '22,000'), (10, '일반펌', '여성일반펌', '100,000'), (10, '일반펌', '남성일반펌', '70,000'), (10, '열펌', '여성열펌', '120,000'), (10, '클리닉', '여성크리닉', '70,000')]
[('3850700', '남성트렌드전문 지아 [화요일휴무]', '4.99', 'https://booking.naver.com/booking/13/bizes/198218/items/3850700?theme=place&area=ple', '안녕하세요 모티브 헤어 지아에요 1:1 맞춤 시술로 손질이 편하며 무드있고 트렌디한 스타일 추천해드릴게요 :] * 당일 예약 시 약간의 대기 시간 발생할 수 있어용 !'), ('2929693', '남자머리 고수 진 [수요일휴무]', '4.97', 'https://booking.naver.com/booking/13/bizes/198218/items/2929693?theme=place&area=ple', '남자머리고수 JIN 디자이너 입니다. 같은 남성으로써 누구보다 남자들의 고민을 디테일하고 섬세하게 해결해 드리는 차분한 감성디자이너 입니다.'), ('3737909', '믿고 찾고 다시보는 다운쌤 [목요일휴무]', '4.92', 'https://booking.naver.com/booking/13/bizes/198218/items/3737909?theme=place&area=ple', 'C컬펌과 컬러재예약이 많은 실장 다운입니다. 친구처럼 고민하며 고객님의 Best 스타일을 찾아드리는 스타일리스트로 믿고 찾고 다시보는 인간미 뿜뿜 쌤입나다.'), ('2929690', '펌신펌왕 리더팀장 유나쌤 [일요일 휴무]', '4.95', 'h